### Segmenting - Part 1

First of all, I am importing the necessary libraries.

In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

Next, I am reading the table from the webpage into a pandas dataframe.

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

This next line of code ignores cells with a borough that is Not assigned.

In [3]:
df = df[df.Borough != 'Not assigned']

Combining neighbourhoods with the same postcode into one row:

In [4]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

Renaming neighbourhoods that are Not assigned:

In [10]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough 
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df.shape

(103, 3)

In [11]:
geo = pd.read_csv('C:/Users/Owner/Downloads/Geospatial_Coordinates.csv')
geo.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
geo_data = pd.merge(df, geo, how='inner', on='PostalCode')
geo_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
geo_grouped = geo_data.groupby('Borough').count()
geo_grouped

,PostalCode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


Run clustering and generate maps

In [23]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Owner\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be UPDATED:

  anaconda                                    custom-py37_1 --> 2019.03-py37_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2019.6.1~ --> pkgs/main::ca-certificates-2019.1.23-0
  openssl            conda-forge::openssl-1.1.1c-hfa6e2cd_0 --> pkgs/main::openssl-1.1.1b-he774522_1

The following packages will be DOWNGRADED:

  certifi                                  2019.6.16-py37_1 --> 2019.3.9-py37_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [26]:
CLIENT_ID = '2VURWDB4ZLRFCTGABYLHTLJNI5HQCLC21Z22K1ABN2C5WERH' # your Foursquare ID
CLIENT_SECRET = 'JUKCBWAWK5NKGH3YP2KZDP53HUEI3ALEZD4VD3UGOKRCO1MO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2VURWDB4ZLRFCTGABYLHTLJNI5HQCLC21Z22K1ABN2C5WERH
CLIENT_SECRET:JUKCBWAWK5NKGH3YP2KZDP53HUEI3ALEZD4VD3UGOKRCO1MO


In [35]:
kclusters = 5

#manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(geo_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 1, 2, 2, 4, 0, 3, 0, 1, 2])

In [41]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(geo_grouped['Latitude'], geo_grouped['Longitude'], geo_grouped['Neighbourhood'], geo_grouped['PostalCode']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-3],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(map_clusters)


map_clusters

IndexError: list index out of range